<a href="https://colab.research.google.com/github/sharathnallaganti/LLM-s/blob/main/S1_Model_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# --- System tools & OCR ---
!apt-get -y install tesseract-ocr poppler-utils > /dev/null

# --- Core Python libs ---
!pip install -q fastapi uvicorn pydantic==2.* \
    sentence-transformers faiss-cpu \
    pdfplumber pypdf python-docx unstructured[all-docs] \
    pillow rapidfuzz numpy pandas tqdm \
    httpx langchain==0.2.* langchain-community llama-index \
    pydantic-settings gradio


In [2]:
!pip install -q transformers accelerate bitsandbytes

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_id = "mistralai/Mistral-7B-Instruct-v0.2"  # small and fast enough
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id, device_map="auto", torch_dtype="auto"
)

llm = pipeline("text-generation", model=model, tokenizer=tokenizer)

def generate(prompt):
    out = llm(prompt, max_new_tokens=200, temperature=0.2, do_sample=False)
    return out[0]["generated_text"]


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Device set to use cpu


In [1]:
from sentence_transformers import SentenceTransformer
import faiss, numpy as np

# 1. Build small test index
texts = ["The Constitution was signed in 1787.",
         "Machine learning uses data to train models.",
         "Tesseract performs OCR on scanned documents."]
model = SentenceTransformer("all-MiniLM-L6-v2")

X = model.encode(texts, normalize_embeddings=True)
index = faiss.IndexFlatIP(X.shape[1])
index.add(np.array(X).astype("float32"))

# 2. Query
query = "When was the Constitution signed?"
q_emb = model.encode([query], normalize_embeddings=True).astype("float32")
scores, idx = index.search(q_emb, k=2)
context = "\n".join(texts[i] for i in idx[0])

# 3. Generate
prompt = f"Answer the question using the context.\n\nContext:\n{context}\n\nQuestion: {query}"
print(generate(prompt))


ModuleNotFoundError: No module named 'faiss'